# let's make a datframe with a column we want to split on, and do it



## libraries

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# initialize list of lists 
data = [['tom', 10, 6,'M'], 
        ['nick', 15,8, 'F'], 
        ['juli', 14,8,'F'],
        ['lolo', 23,10, 'M'], 
        ['mimi',66,10, 'M'],
        ['tim', 11,32, 'M'], 
        ['dick', 15,23, 'F'], 
        ['june', 24,23, 'F'],
        ['leem', 24, 23, 'M'],
        ['mia',69,23, 'M']] 
  
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns=['Name', 'Age', 'nonsense', 'Sex']) 
  
# print dataframe. 
df 

In [ ]:
df.Age.describe()

In [ ]:
def split_frame_half_balanced_by_column(frame, column):
    """This is useful in cases where you want to split on a columns
    with continous values e.g. age."""
    df = frame.sort_values(column).reset_index()
    rng_even = range(0, len(df),2)
    rng_odd =  range(1, len(df),2)
    even_rows = df.iloc[rng_even]
    odd_rows = df.iloc[rng_odd]
    return even_rows, odd_rows

In [ ]:
split_frame_half_balanced_by_column(df, 'Age')[0]

In [ ]:
df['Sex'].unique()

# rng_even = range(0, len(df),2)
# rng_odd =  range(1, len(df),2)
# even_rows = df.iloc[rng_even]
# odd_rows = df.iloc[rng_odd]
# odd_rows

In [ ]:
pd.cut(df,3)

In [ ]:

df_1=df.groupby('Age', as_index=False, group_keys=False).apply(lambda x: x.sample(n=2))
df_2 = df[~df.index.isin(df_1.index)]

In [ ]:
df_1=df.groupby('Age').apply(lambda x: x.sample(n=2))

In [ ]:
df_1